In [1]:
from importlib import reload
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('./dracula.mplstyle')
import logging
logging.basicConfig()
LOG = logging.getLogger(__name__)
LOG.setLevel(logging.INFO)
LOG.info('Hello from notebook')

from pydataset import data
df = data('diamonds').reset_index().set_index('index')

import frankenfit as ff
reload(ff)

logging.getLogger('frankenfit').setLevel(logging.INFO)

INFO:__main__:Hello from notebook


In [17]:
client.shutdown()

In [18]:
cluster.close()

In [2]:
from dask import distributed
cluster = distributed.LocalCluster(
    n_workers=4, threads_per_worker=2, host="devbox.homenet.thebanes.org",
    scheduler_port=0, dashboard_address=':0'
)
print(cluster.dashboard_link)
client = distributed.Client(cluster)
client

#cluster = distributed.LocalCluster(n_workers=1, threads_per_worker=1)
#client = distributed.Client(cluster)
#client = distributed.Client()
#client

http://10.0.2.82:37343/status


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://10.0.2.82:37343/status,
Dashboard: http://10.0.2.82:37343/status,Workers: 4
Total threads: 8,Total memory: 23.47 GiB
Status: running,Using processes: True
Comm: tcp://10.0.2.82:34903,Workers: 4
Dashboard: http://10.0.2.82:37343/status,Total threads: 8
Started: Just now,Total memory: 23.47 GiB
Comm: tcp://10.0.2.82:41279,Total threads: 2
Dashboard: http://10.0.2.82:42365/status,Memory: 5.87 GiB
Nanny: tcp://10.0.2.82:45075,


In [3]:
dask = ff.DaskBackend(client)
dask

DaskBackend(address='tcp://10.0.2.82:34903')

In [4]:
p = (
    ff.ReadDataset('./diamonds.csv', format='csv', index_col='index')
    .then()  # something wrong with return type of then() here; type params not bound?
    .z_score(["price"])
    .clip(["price"], lower=-2, upper=2)
)
p

DataFramePipeline(tag='DataFramePipeline#1', transforms=[ReadDataset(tag='ReadDataset#1', paths=['./diamonds.csv'], columns=None, format='csv', filter=None, index_col='index', dataset_kwargs=None, scanner_kwargs=None), ZScore(tag='ZScore#1', cols=['price'], w_col=None), Clip(tag='Clip#1', cols=['price'], upper=2, lower=-2)])

In [5]:
p.apply(backend=dask).result().head()

,carat,cut,color,clarity,depth,table,price,x,y,z
index,,,,,,,,,,
1,0.23,Ideal,E,SI2,61.5,55.0,-0.904087,3.95,3.98,2.43
2,0.21,Premium,E,SI1,59.8,61.0,-0.904087,3.89,3.84,2.31
3,0.23,Good,E,VS1,56.9,65.0,-0.903836,4.05,4.07,2.31
4,0.29,Premium,I,VS2,62.4,58.0,-0.902081,4.20,4.23,2.63
5,0.31,Good,J,SI2,63.3,58.0,-0.901831,4.34,4.35,2.75


In [5]:
tf = p.fit(backend=dask)
tf.state()

DummyFuture(obj=[FitTransform(resolved_transform=ReadDataset(tag='ReadDataset#1', paths=['./diamonds.csv'], columns=None, format='csv', filter=None, index_col='index', dataset_kwargs=None, scanner_kwargs=None), state=<class 'frankenfit.backend.DummyFuture'>, bindings={}), FitTransform(resolved_transform=ZScore(tag='ZScore#1', cols=['price'], w_col=None), state=<class 'frankenfit.backend.DaskFuture'>, bindings={}), FitTransform(resolved_transform=Clip(tag='Clip#1', cols=['price'], upper=2, lower=-2), state=<class 'frankenfit.backend.DummyFuture'>, bindings={})])

In [6]:
tf.apply(backend=dask).result().head()

,carat,cut,color,clarity,depth,table,price,x,y,z
index,,,,,,,,,,
1,0.23,Ideal,E,SI2,61.5,55.0,-0.904087,3.95,3.98,2.43
2,0.21,Premium,E,SI1,59.8,61.0,-0.904087,3.89,3.84,2.31
3,0.23,Good,E,VS1,56.9,65.0,-0.903836,4.05,4.07,2.31
4,0.29,Premium,I,VS2,62.4,58.0,-0.902081,4.20,4.23,2.63
5,0.31,Good,J,SI2,63.3,58.0,-0.901831,4.34,4.35,2.75


In [7]:
FEATURES = ["carat", "x", "y", "z", "depth", "table"]

def bake_features(cols):
    return (
        ff.DataFramePipeline()
        .print(fit_msg=f"Baking: {cols}")
        .winsorize(cols, limit=0.05)
        .z_score(cols)
        .impute_constant(cols, 0.0)
        .clip(cols, upper=2, lower=-2)
    )

ff.ReadDataset('./diamonds.csv', format='csv', index_col='index').then(bake_features(FEATURES)).apply().head()

Baking: ['carat', 'x', 'y', 'z', 'depth', 'table']


,carat,cut,color,clarity,depth,table,price,x,y,z
index,,,,,,,,,,
1,-1.131845,Ideal,E,SI2,-0.207349,-1.206255,326,-1.345194,-1.345905,-1.343759
2,-1.131845,Premium,E,SI1,-1.663782,1.835534,326,-1.345194,-1.345905,-1.343759
3,-1.131845,Good,E,VS1,-2.000000,1.835534,327,-1.345194,-1.345905,-1.343759
4,-1.131845,Premium,I,VS2,0.563704,0.314640,334,-1.345194,-1.345905,-1.343759
5,-1.108324,Good,J,SI2,1.334757,0.314640,335,-1.297969,-1.298397,-1.190838


In [8]:
(
    ff.ReadDataset('./diamonds.csv', format='csv', index_col='index')
    .then(bake_features(FEATURES))
    .apply(backend=dask).result().head()
)

Baking: ['carat', 'x', 'y', 'z', 'depth', 'table']


,carat,cut,color,clarity,depth,table,price,x,y,z
index,,,,,,,,,,
1,-1.131845,Ideal,E,SI2,-0.207349,-1.206255,326,-1.345194,-1.345905,-1.343759
2,-1.131845,Premium,E,SI1,-1.663782,1.835534,326,-1.345194,-1.345905,-1.343759
3,-1.131845,Good,E,VS1,-2.000000,1.835534,327,-1.345194,-1.345905,-1.343759
4,-1.131845,Premium,I,VS2,0.563704,0.314640,334,-1.345194,-1.345905,-1.343759
5,-1.108324,Good,J,SI2,1.334757,0.314640,335,-1.297969,-1.298397,-1.190838


In [9]:
from sklearn.linear_model import LinearRegression

FEATURES = ["carat", "x", "y", "z", "depth", "table"]

def bake_features(cols):
    return (
        ff.DataFramePipeline()
        .print(fit_msg=f"Baking: {cols}")
        .winsorize(cols, limit=0.05)
        .z_score(cols)
        .impute_constant(cols, 0.0)
        .clip(cols, upper=2, lower=-2)
    )

complex_pipeline = (
    ff.DataFramePipeline()[FEATURES + ["{response_col}"]]
    .copy("{response_col}", "{response_col}_train")
    .winsorize("{response_col}_train", limit=0.05)
    .pipe(["carat", "{response_col}_train"], np.log1p)
    .then(bake_features(FEATURES))
    #.if_hyperparam_is_true("bake_features", bake_features(FEATURES))
    .sk_learn(
        LinearRegression,
        # x_cols=["carat", "depth", "table"],
        x_cols=ff.HP("predictors"),
        response_col="{response_col}_train",
        hat_col="{response_col}_hat",
        class_params={"fit_intercept": True},
    )
    # transform {response_col}_hat from log-dollars back to dollars
    .copy("{response_col}_hat", "{response_col}_hat_dollars")
    .pipe("{response_col}_hat_dollars", np.expm1)
)

complex_pipeline.hyperparams()

{'predictors', 'response_col'}

In [10]:
bindings = {"response_col": "price", "bake_features": True, "predictors": FEATURES}

In [11]:
read_diamonds = ff.ReadDataset('./diamonds.csv', format='csv', index_col='index')

In [15]:
#ff.ReadDataset('./diamonds.csv', format='csv', index_col='index').then(complex_pipeline).fit(bindings=bindings)
read_diamonds.then(complex_pipeline).fit(bindings=bindings).apply().head()

Baking: ['carat', 'x', 'y', 'z', 'depth', 'table']


,carat,x,y,z,depth,table,price,price_train,price_hat,price_hat_dollars
index,,,,,,,,,,
1,-1.251924,-1.345194,-1.345905,-1.343759,-0.207349,-1.206255,326,6.300786,6.480260,651.140243
2,-1.251924,-1.345194,-1.345905,-1.343759,-1.663782,1.835534,326,6.300786,6.419302,612.574861
3,-1.251924,-1.345194,-1.345905,-1.343759,-2.000000,1.835534,327,6.300786,6.416375,610.781453
4,-1.251924,-1.345194,-1.345905,-1.343759,0.563704,0.314640,334,6.300786,6.462834,639.874609
5,-1.218600,-1.297969,-1.298397,-1.190838,1.334757,0.314640,335,6.300786,6.549435,697.848961


In [12]:
fit = read_diamonds.then(complex_pipeline).fit(bindings=bindings, backend=dask)
fit

Baking: ['carat', 'x', 'y', 'z', 'depth', 'table']


FitBasePipeline(resolved_transform=DataFramePipeline(tag='DataFramePipeline#8', transforms=[ReadDataset(tag='ReadDataset#4', paths=['./diamonds.csv'], columns=None, format='csv', filter=None, index_col='index', dataset_kwargs=None, scanner_kwargs=None), Select(tag='Select#1', cols=['carat', 'x', 'y', 'z', 'depth', 'table', '{response_col}']), Copy(tag='Copy#1', cols=['{response_col}'], dest_cols=['{response_col}_train']), Winsorize(tag='Winsorize#3', cols=['{response_col}_train'], limit=0.05), Pipe(tag='Pipe#1', cols=['carat', '{response_col}_train'], apply_fun=<ufunc 'log1p'>), Print(tag='Print#3', fit_msg="Baking: ['carat', 'x', 'y', 'z', 'depth', 'table']", apply_msg=None, dest=None), Winsorize(tag='Winsorize#4', cols=['carat', 'x', 'y', 'z', 'depth', 'table'], limit=0.05), ZScore(tag='ZScore#4', cols=['carat', 'x', 'y', 'z', 'depth', 'table'], w_col=None), ImputeConstant(tag='ImputeConstant#3', cols=['carat', 'x', 'y', 'z', 'depth', 'table'], value=0.0), Clip(tag='Clip#4', cols=['c

In [13]:
fit.state()

DummyFuture(obj=[FitTransform(resolved_transform=ReadDataset(tag='ReadDataset#4', paths=['./diamonds.csv'], columns=None, format='csv', filter=None, index_col='index', dataset_kwargs=None, scanner_kwargs=None), state=<class 'frankenfit.backend.DummyFuture'>, bindings={'response_col': 'price', 'bake_features': True, 'predictors': ['carat', 'x', 'y', 'z', 'depth', 'table']}), FitTransform(resolved_transform=Select(tag='Select#1', cols=['carat', 'x', 'y', 'z', 'depth', 'table', 'price']), state=<class 'frankenfit.backend.DummyFuture'>, bindings={'response_col': 'price', 'bake_features': True, 'predictors': ['carat', 'x', 'y', 'z', 'depth', 'table']}), FitTransform(resolved_transform=Copy(tag='Copy#1', cols=['price'], dest_cols=['price_train']), state=<class 'frankenfit.backend.DummyFuture'>, bindings={'response_col': 'price', 'bake_features': True, 'predictors': ['carat', 'x', 'y', 'z', 'depth', 'table']}), FitTransform(resolved_transform=Winsorize(tag='Winsorize#3', cols=['price_train'],

In [14]:
fit.apply(backend=dask).result().head()

,carat,x,y,z,depth,table,price,price_train,price_hat,price_hat_dollars
index,,,,,,,,,,
1,-1.251924,-1.345194,-1.345905,-1.343759,-0.207349,-1.206255,326,6.300786,6.480260,651.140243
2,-1.251924,-1.345194,-1.345905,-1.343759,-1.663782,1.835534,326,6.300786,6.419302,612.574861
3,-1.251924,-1.345194,-1.345905,-1.343759,-2.000000,1.835534,327,6.300786,6.416375,610.781453
4,-1.251924,-1.345194,-1.345905,-1.343759,0.563704,0.314640,334,6.300786,6.462834,639.874609
5,-1.218600,-1.297969,-1.298397,-1.190838,1.334757,0.314640,335,6.300786,6.549435,697.848961


In [15]:
fit.apply().head()

,carat,x,y,z,depth,table,price,price_train,price_hat,price_hat_dollars
index,,,,,,,,,,
1,-1.251924,-1.345194,-1.345905,-1.343759,-0.207349,-1.206255,326,6.300786,6.480260,651.140243
2,-1.251924,-1.345194,-1.345905,-1.343759,-1.663782,1.835534,326,6.300786,6.419302,612.574861
3,-1.251924,-1.345194,-1.345905,-1.343759,-2.000000,1.835534,327,6.300786,6.416375,610.781453
4,-1.251924,-1.345194,-1.345905,-1.343759,0.563704,0.314640,334,6.300786,6.462834,639.874609
5,-1.218600,-1.297969,-1.298397,-1.190838,1.334757,0.314640,335,6.300786,6.549435,697.848961


In [16]:
read_diamonds.then(complex_pipeline).apply(bindings=bindings, backend=dask).result().head()

Baking: ['carat', 'x', 'y', 'z', 'depth', 'table']


,carat,x,y,z,depth,table,price,price_train,price_hat,price_hat_dollars
index,,,,,,,,,,
1,-1.251924,-1.345194,-1.345905,-1.343759,-0.207349,-1.206255,326,6.300786,6.480260,651.140243
2,-1.251924,-1.345194,-1.345905,-1.343759,-1.663782,1.835534,326,6.300786,6.419302,612.574861
3,-1.251924,-1.345194,-1.345905,-1.343759,-2.000000,1.835534,327,6.300786,6.416375,610.781453
4,-1.251924,-1.345194,-1.345905,-1.343759,0.563704,0.314640,334,6.300786,6.462834,639.874609
5,-1.218600,-1.297969,-1.298397,-1.190838,1.334757,0.314640,335,6.300786,6.549435,697.848961


In [ ]:
# Good TODOs:
# [x] Optimize StatelessTransform.fit() not to submit/always use DummyBackend?
# Impure Transforms like data readers.
# [x] BasePipeline.fit() and FitTransform[BasePipeline, ...] should chain backend submissions, like BasePipeline.apply()
# Backend.fit()/apply() methods. apply() overloaded for FitTransform, StatelessTransform, BasePipeline.
# ForBindings: combine_fun param, maybe generic in DataResult; DataFramePipeline expects specialized for DataFrames
# Start parallelizing appropriate transforms: ForBindings, Join, GroupByCols, GroupByBindings